In [2]:
import pandas as pd

In [80]:
bookings_df1 = """
503200025365
503200025390
503200025390
503200025403
503200025705
503200025705
503200025845""".split()

bookings_df2 = """
503200025365
503200025390.0
503200025390
503200025403
503200025403
503200025403
503200025705
503200025705
""".split()

containers_df2 ="""
DFSU6896599
EISU9363352
TGBU6778406
EGHU1003711
EGHU1019723
RTYH1256789
QWER1234567
TYUI2536890
""".split()

nwt = """
9990
8344
14692
14040
11520
12850
12720
""".split()
nwt2 = """
13.79
12.22
18.39
17.67
15.15
16.47
""".split()


In [96]:
df1 = pd.DataFrame({'BOOKING': bookings_df1})
df2 = pd.DataFrame({'BOOKING': bookings_df2})

df_concat = pd.concat([df1, df2])
booking_array = df_concat['BOOKING'].unique()

In [102]:
df2.astype(str).replace('.0', '')

,BOOKING
0,503200025365
1,503200025390.0
2,503200025390
3,503200025403
4,503200025403
5,503200025403
6,503200025705
7,503200025705


In [143]:
def egl_si():
    file_path = r"docs\si\RDX1 005S SEGOT.xlsm"
    df = pd.read_excel(file_path, sheet_name='Output', header=8, usecols=("A,C,E,I:K"))
    df = df.rename(columns={
        '  CONTAINER NO. '      : 'CONTAINER',
        ' PACKAGE'              : 'PKG',
        '    GWT '              : 'GWT',
        '    B/L NO.  '         : 'BOOKING',
        ' MRN No / Q-status '   : 'MRN',
        ' PARTNER'              : 'MLO'
        })
    df = df.drop(index=0)
    df = df.dropna(axis='index', how='all')
    df[['CONTAINER', 'TYPE']] = df['CONTAINER'].str.split(expand=True)
    df['BOOKING'] = df['BOOKING'].str.replace(' ', '')
    df.loc[:,'BOOKING'] = df['BOOKING'].astype(str).replace(r'\.0*', '')
    df = df.reset_index(drop=True)
    df = df.sort_values('BOOKING')
    
    return df

In [144]:
import xlwings as xw

In [145]:

def load_df():
    wb = xw.Book(r"docs\0114_Bokningsblad_ANINA_23004_SEGOT-ursprunglig.xlsb")
    sheet = wb.sheets('INFO')
    data_table = sheet.range('A4').expand()
    df = sheet.range(data_table).options(pd.DataFrame, index=False, header=True).value
    df = df.loc[(df.MLO == 'EVER') & (df['LOAD STATUS'] != 'MT'), :]
    df = df[['BOOKING NUMBER', 'MLO', 'CONTAINER', 'ISO TYPE', 'NET WEIGHT', 'LOAD STATUS', 'VGM', 'OOG', 'REMARK', 'IMDG', 'UNNR', 'MRN', 'TEMP', 'PACKAGES']]
    df.insert(0, 'COMMENT', 'Bokningsblad')
    df.loc[:,'BOOKING NUMBER'] = df['BOOKING NUMBER'].astype(str).replace('.0', '', regex=False)
    df['MLO'] = None
    df['CONTAINER'] = None
    df['ISO TYPE'] = None
    df['NET WEIGHT'] = None
    df = df.sort_values('BOOKING NUMBER')
    
    return df

In [146]:
df1 = load_df()
df2 = egl_si()

df1 = df1.rename(columns={'BOOKING NUMBER': 'BOOKING'})
df1['BOOKING'] = df1['BOOKING'].astype(str)
df1['BOOKING'] = df1['BOOKING'].str.replace('.0', '', regex=False)

df_concat = pd.concat([df1, df2])
booking_array = df_concat['BOOKING'].unique()

df1['COMMENT'] = "Bokningsblad"

Output


In [201]:
def booking_count(df1: pd.DataFrame, df2: pd.DataFrame, booking: str) -> int:
    df1_count, df2_count = 0, 0

    if booking in df1['BOOKING'].values:
        df1_count = df1['BOOKING'].value_counts()[booking]
    if booking in df2['BOOKING'].values:
        df2_count = df2['BOOKING'].value_counts()[booking]
        
    return max(df1_count, df2_count)

In [202]:
"""string = ""
for booking in booking_array:
    string += f'{booking},' * booking_count(df1, df2, booking)
"""

'string = ""\nfor booking in booking_array:\n    string += f\'{booking},\' * booking_count(df1, df2, booking)\n'

In [203]:

string = ""
string += ''.join(f'{booking}, ' * booking_count(df1, df2, booking) for booking in booking_array)

In [204]:
booking_list = ""
booking_list = string.split(',')

In [205]:
booking_list.pop()
booking_list

['503200025845',
 ' 503200026302',
 ' 503200026302',
 ' 503200026302',
 ' 503200026302',
 ' 503200026302',
 ' 503200026302',
 ' 503200026302',
 ' 503200026302',
 ' 503300000119',
 ' 503300000119',
 ' 503300000119',
 ' 503300000674',
 ' 503300000674',
 ' 503300000674',
 ' 503300000674',
 ' 503300000674',
 ' 503300000712',
 ' 510300002157',
 ' 503200025365',
 ' 503200025390',
 ' 503200025390',
 ' 503200025403',
 ' 503200025403',
 ' 503200025403',
 ' 503200025403',
 ' 503200025403',
 ' 503200025403',
 ' 503200025403',
 ' 503200025403',
 ' 503200025403',
 ' 503200025403',
 ' 503200025705',
 ' 503200025705',
 ' 503200025705',
 ' 503200025705',
 ' 503200025705',
 ' 503200025705',
 ' 503200025705',
 ' 503200025705',
 ' 503200025896',
 ' 503200026001',
 ' 503200026001',
 ' 503200026001',
 ' 503200026248',
 ' 503200026256',
 ' 503200026281',
 ' 503200026311',
 ' 503200026311',
 ' 503200026345',
 ' 503300000054',
 ' 503300000054',
 ' 503300000054',
 ' 503300000071',
 ' 503300000186',
 ' 50330000

In [210]:
df2['BOOKING'][0]

'503200025365'

In [171]:
counter_df1 = 0
counter_df2 = 0

for num, booking in enumerate(booking_list):
    pass


,BOOKING
503200025390,"[DFSU6896599, EISU9363352, TGBU6778406, EGHU10..."
503300000186,"[EGHU1019723, RTYH1256789]"


In [141]:
df3, df4 = "", ""

df3 = pd.DataFrame({'BOOKING': ['503200025845',
 '503200026302',
 '503200026302',
 '503200026302',
 '503200026302',
 '503200026302',
 '503200026302',
 '503200026302',
 '503200026302',
 '503300000119',
 '503300000119',
 '503300000119',]})

df4 = pd.DataFrame({'BOOKING': ['503200025845',
 '503200026302',
 '503200026302',
 '503200026302',
 '503200026302',
 '503200026302',
 '503200026302',
 '503200026302',
 '503200026302',
 '503300000119',
 '503300000119',
 '503300000120',]})

pd.merge(df3, df4, how='outer', on='BOOKING')

,BOOKING
0,503200025845
1,503200026302
2,503200026302
3,503200026302
4,503200026302
...,...
67,503300000119
68,503300000119
69,503300000119
70,503300000119


In [142]:
df1

,COMMENT,BOOKING,MLO,CONTAINER,ISO TYPE,NET WEIGHT,LOAD STATUS,VGM,OOG,REMARK,IMDG,UNNR,MRN,TEMP,PACKAGES
21,Bokningsblad,503200025845,None,None,None,None,LA,24885.0,None,None,None,None,None,None,26.0
29,Bokningsblad,503200026302,None,None,None,None,LA,30075.0,None,None,None,None,None,None,104.0
30,Bokningsblad,503200026302,None,None,None,None,LA,30225.0,None,None,None,None,None,None,104.0
31,Bokningsblad,503200026302,None,None,None,None,LA,30200.0,None,None,None,None,None,None,104.0
32,Bokningsblad,503200026302,None,None,None,None,LA,29050.0,None,None,None,None,None,None,104.0
33,Bokningsblad,503200026302,None,None,None,None,LA,30025.0,None,None,None,None,None,None,104.0
34,Bokningsblad,503200026302,None,None,None,None,LA,30025.0,None,None,None,None,None,None,104.0
35,Bokningsblad,503200026302,None,None,None,None,LA,30100.0,None,None,None,None,None,None,104.0
36,Bokningsblad,503200026302,None,None,None,None,LA,30025.0,None,None,None,None,None,None,104.0
46,Bokningsblad,503300000119,None,None,None,None,LA,26850.0,None,None,None,None,None,None,26.0
